In [ ]:
import datetime

group_activities_by_name = ["2000", "1000", "400", "200", "Ritmado", "Rodagem", "Without Name Match"] #This List can be used to group activities by name... in my case I use custom workouts name to my activities like "3x2000 or 20x200"... can be left empty if you use the default garmin activity name.
group_activities_by_lap_distance = [1000, 800, 600, 400, 200] #This list can be used to group activities with laps with the same distance and the pace below the next variable.
group_activities_by_lap_speed = datetime.time(0, 5, 0) #This is the pace filter used to group activities by distance
lap_distance_precision = 50 #This is the distance precision to group, with the number 50 laps from 950 to 1050 will be counted as 1000 laps. Useful if you don't use custom workouts with predefined distances.
original_lap_filter_list = ["None"] + [1000, 800, 600, 400, 200] #Lista to filter the laps from the current activity. This way you can select a training that you made and watch only the laps from X distance, or select None and watch all the laps.

In [ ]:
from ipywidgets import interact, fixed, Layout, interactive
from garmindb import ConfigManager, GarminConnectConfigManager
from garmindb.garmindb import GarminDb, Attributes, ActivitiesDb, Activities, ActivityLaps
from collections import ChainMap
import fitfile
from fitfile import Distance
import pandas as pd
import datetime
import math

gc_config = GarminConnectConfigManager()
db_params_dict = ConfigManager.get_db_params()
garmin_db = GarminDb(db_params_dict)
garmin_act_db = ActivitiesDb(db_params_dict)
measurement_system = Attributes.measurements_type(garmin_db)
unit_strings = fitfile.units.unit_strings[measurement_system]
distance_units = unit_strings[fitfile.units.UnitTypes.distance_long]
altitude_units = unit_strings[fitfile.units.UnitTypes.altitude]
temp_units = unit_strings[fitfile.units.UnitTypes.tempurature]


group_activities_by_lap_distance_converted = [Distance.from_meters_or_feet(distance).kms_or_miles() for distance in group_activities_by_lap_distance]
activities_dict = {key: [] for key in group_activities_by_name + group_activities_by_lap_distance}
current_selected_lap = "None"
current_selected_pace = "None"
Treino = ['All'] + list(activities_dict.keys())
activities_list = []
complete_laps_list = []
custom_layout = Layout(width='max-content')     
custom_style = {'description_width': '100px'}

lap_distance_precision = Distance.from_meters_or_feet(lap_distance_precision).kms_or_miles()

class CustomActivity(object):
    
    def __init__(self, id, name, date):
        self.id = id
        self.name = name
        self.date = date
    
    def __repr__(self):
        return f"{self.date} - {self.name} - {self.id}"

def get_pace_list():
    # Initialize the start and end times in seconds
    start_time = 3 * 60  # 3 minutes in seconds
    end_time = 7 * 60   # 10 minutes in seconds

    # Create an empty list to store the formatted times
    time_list = []

    # Iterate from start_time to end_time in 1-second increments
    for time_in_seconds in range(start_time, end_time + 1):
        # Calculate minutes and seconds
        minutes = time_in_seconds // 60
        seconds = time_in_seconds % 60

        # Format the time as a string and append to the list
        time_str = f"{minutes:02}:{seconds:02}"
        time_list.append(time_str)

    # return the list of formatted times
    return time_list[::10]

def convert_to_time(pace_filter: str) -> datetime.time:
    try:
        time_list = pace_filter.split(":")
        return datetime.time(0, int(time_list[0]), int(time_list[1]))
    except:
        return datetime.time.max

def make_delta(total_time: datetime.time) -> datetime.timedelta:
    hours = total_time.hour
    minutes = total_time.minute
    seconds = total_time.second + (total_time.microsecond / 1000000)
    return datetime.timedelta(hours=hours, minutes=minutes, seconds=seconds)

def format_df(original_df):
    try:
        df = original_df.copy()
        df["time"] = df["time"].apply(lambda x: round_up_miliseconds(x).strftime("%M:%S"))
        df["pace"] = df["pace"].apply(lambda x: round_up_miliseconds(x).strftime("%M:%S"))
        df["avg_pace"] = df["avg_pace"].apply(lambda x: round_up_miliseconds(x).strftime("%M:%S"))
        df = df[["lap", "distance", "time", "pace", "avg_pace", "dif_pace", "speed", "avg_hr", "max_hr", "ascent", "descent"]]
        df = df.style.format(precision=1)
        return df
    except:
        return "Nenhum Dado Detectado"

def remove_duplicates_from_list(my_list):
    return list(dict.fromkeys(my_list))

def remove_duplicates(objects, key=lambda x: x):
    seen = set()
    unique_objects = []
    for obj in objects:
        obj_key = key(obj)
        if obj_key not in seen:
            seen.add(obj_key)
            unique_objects.append(obj)
    return unique_objects

def round_up_miliseconds(my_time):
    microseconds = my_time.microsecond
    if microseconds > 0:
        # Calculate the number of microseconds to round up to the nearest second
        microseconds_to_round = 1000000 - microseconds

        # Create a timedelta with the microseconds to round up
        rounding_delta = datetime.timedelta(microseconds=microseconds_to_round)

        # Add the rounding delta to the original time
        my_time = (datetime.datetime.combine(datetime.date(1, 1, 1), my_time) + rounding_delta).time()

    return my_time
    # Format the time as a string

def convert_time_to_seconds(time_variable: datetime.time) -> int:
    return math.ceil((datetime.datetime.combine(datetime.datetime.min, time_variable) - datetime.datetime.min).total_seconds())
    
def time_dif_in_seconds(time_1: datetime.time, time_2: datetime.time) -> int:
    return convert_time_to_seconds(time_1) - convert_time_to_seconds(time_2)

def extract_main_laps(laps, laps_filter, pace_filter):
    try:
        main_laps = []
        for lap in laps:
            if (laps_filter == "None" or math.isclose(lap.distance, Distance.from_meters_or_feet(laps_filter).kms_or_miles(), rel_tol=lap_distance_precision)):
                lap.pace = fitfile.conversions.perhour_speed_to_pace(lap.avg_speed)
                if pace_filter == "None" or lap.pace < convert_to_time(pace_filter):
                    custom_lap_dict = {
                        "lap": lap.lap,
                        "distance": lap.distance,
                        "time": lap.moving_time,
                        "pace": lap.pace,
                        "speed": lap.avg_speed,
                        "avg_hr": lap.avg_hr,
                        "max_hr": lap.max_hr,
                        "ascent": lap.ascent,
                        "descent": lap.descent,
                    }
                    main_laps.append(custom_lap_dict)
        return main_laps
    except:
        return []

def get_laps_df(activity_obj: CustomActivity, laps_filter="None", pace_filter="None"):
    laps = ActivityLaps.get_activity(garmin_act_db, activity_obj.id)
    main_laps_list = extract_main_laps(laps, laps_filter=laps_filter, pace_filter=pace_filter)
    if len(main_laps_list) == 0:
        return "No Laps Found With Current Filters"
    laps_df = pd.DataFrame(main_laps_list)
    total_laps_time = (datetime.datetime.min + laps_df["time"].apply(lambda x: make_delta(x)).sum()).time()
    total_laps_distance = laps_df["distance"].sum()
    total_laps_in_hour = convert_time_to_seconds(total_laps_time) / 3600
    medium_pace = fitfile.conversions.perhour_speed_to_pace(total_laps_distance / total_laps_in_hour)
    laps_df["avg_pace"] = medium_pace
    laps_df["dif_pace"] = laps_df.apply(lambda x: time_dif_in_seconds(x["avg_pace"],x["pace"]), axis=1)
    return laps_df

def select_training_filter(treino):
    if treino == 'All':
        tmp = list(ChainMap(*[activities_dict.get(key) for key in activities_dict.keys()]))
        tmp = sorted(tmp, key=lambda x: x.date, reverse=True)
        tmp = remove_duplicates(tmp, key=lambda obj: obj.id)
        activity = tmp
    else:
        activity = activities_dict.get(treino)
    activity_widget = interactive(select_activity, activity=activity)
    activity_widget.children[0].description = "Activity"
    activity_widget.children[0].style = custom_style
    activity_widget.children[0].layout = custom_layout
    display(activity_widget)

def select_activity(activity):
    global current_selected_lap
    lap_filter_list = original_lap_filter_list.copy()
    if current_selected_lap != "None":
        lap_filter_list.insert(0, current_selected_lap)
    lap_filter_list = remove_duplicates_from_list(lap_filter_list)
    lap_filter_widget = interactive(select_lap_filter, lap_filter=lap_filter_list, activity=fixed(activity))
    lap_filter_widget.children[0].description = "Lap Filter"
    lap_filter_widget.children[0].style = custom_style
    lap_filter_widget.children[0].layout = custom_layout
    display(lap_filter_widget)

def select_lap_filter(lap_filter, activity):
    global current_selected_lap, current_selected_pace
    current_selected_lap = lap_filter
    pace_filter_list = original_pace_filter_list
    if current_selected_pace != "None":
        pace_filter_list.insert(0, current_selected_pace)
    pace_filter_list = remove_duplicates_from_list(pace_filter_list)
    pace_filter_widget = interactive(select_pace_filter, pace_filter=pace_filter_list, activity=fixed(activity), lap_filter=fixed(lap_filter))
    pace_filter_widget.children[0].description = "Pace Filter"
    pace_filter_widget.children[0].style = custom_style
    pace_filter_widget.children[0].layout = custom_layout
    display(pace_filter_widget)

def select_pace_filter(pace_filter, activity, lap_filter):
    global current_selected_pace
    current_selected_pace = pace_filter
    df = get_laps_df(activity, lap_filter, pace_filter)
    display(format_df(df))

def load_activities_list():
    global activities_list, activities_dict
    activities_list = Activities.get_by_sport(garmin_act_db, "running")
    activities_list.reverse()
    for activity in activities_list:
        if len(group_activities_by_name) > 0:
            matches_by_name = [name for name in group_activities_by_name if name in activity.name]
            if len(matches_by_name) > 0:
                for key in matches_by_name:
                    activities_dict[key].append(CustomActivity(activity.activity_id, activity.name, activity.start_time.date()))
            else:
                key = "Without Name Match"
                activities_dict[key].append(CustomActivity(activity.activity_id, activity.name, activity.start_time.date()))
        if len(group_activities_by_lap_distance) > 0:
            laps = ActivityLaps.get_activity(garmin_act_db, activity.activity_id)
            complete_laps_list.extend(laps)
            for lap in laps:
                lap.avg_pace = fitfile.conversions.perhour_speed_to_pace(lap.avg_speed)
            for key, lap_search_distance in zip(group_activities_by_lap_distance, group_activities_by_lap_distance_converted):
                if any([lap for lap in laps if lap.distance is not None and math.isclose(lap.distance, lap_search_distance, rel_tol=lap_distance_precision) and lap.avg_pace < group_activities_by_lap_speed]):
                    activities_dict[key].append(CustomActivity(activity.activity_id, activity.name, activity.start_time.date()))

load_activities_list()
original_pace_filter_list = ["None"] + get_pace_list()
activities_filter_widget = interactive(select_training_filter, treino=Treino)
activities_filter_widget.children[0].description = "Activities Filter"
activities_filter_widget.children[0].style = custom_style
activities_filter_widget.children[0].layout = custom_layout
display(activities_filter_widget)